In [1]:
%matplotlib inline
import os
from glob import glob

import numpy as np
import xarray as xr

from datetime import datetime

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import cartopy
import cartopy.crs as ccrs
import cmocean

import dask

Define a function to fix the plot grid for plotting global maps; this might not be necessary for regional plots, depending on the region.

In [2]:
def adjust_pop_grid(tlon,tlat,field):
    nj = tlon.shape[0]
    ni = tlon.shape[1]
    xL = int(ni/2 - 1)
    xR = int(xL + ni)

    tlon = np.where(np.greater_equal(tlon,min(tlon[:,0])),tlon-360.,tlon)
    lon  = np.concatenate((tlon,tlon+360.),1)
    lon = lon[:,xL:xR]

    if ni == 320:
        lon[367:-3,0] = lon[367:-3,0]+360.
    lon = lon - 360.
    lon = np.hstack((lon,lon[:,0:1]+360.))
    if ni == 320:
        lon[367:,-1] = lon[367:,-1] - 360.

    #-- trick cartopy into doing the right thing:
    #   it gets confused when the cyclic coords are identical
    lon[:,0] = lon[:,0]-1e-8

    #-- periodicity
    lat  = np.concatenate((tlat,tlat),1)
    lat = lat[:,xL:xR]
    lat = np.hstack((lat,lat[:,0:1]))

    field = np.ma.concatenate((field,field),1)
    field = field[:,xL:xR]
    field = np.ma.hstack((field,field[:,0:1]))
    field = np.ma.masked_where(np.isnan(field),field)

    return lon,lat,field

In [3]:
case = 'g.e11.G.T62_t12.eco.006'
droot = f'/glade/scratch/mclong/hi-res-eco/{case}/ocn/hist'
files = sorted(glob(f'{droot}/{case}.pop.h.????-??-??.nc'))
len(files)

365

In [4]:
clobber = False
os.makedirs('figs', exist_ok=True)
def plot(file_in,i):

    plot_name = f'figs/{i:03d}.png'
    if os.path.exists(plot_name) and not clobber:
        return 
    
    #--- patch from https://github.com/SciTools/cartopy/issues/1120
    from matplotlib.axes import Axes
    from cartopy.mpl.geoaxes import GeoAxes
    GeoAxes._pcolormesh_patched = Axes.pcolormesh
    
    keep_vars = ['O2','TLAT','TLONG']
    
    datestr = file_in[file_in.find('h.')+2:file_in.find('h.')+12]
    
    with xr.open_dataset(file_in,decode_times=False,decode_coords=False) as ds:
        ds = ds.drop([v for v in ds.variables if v not in keep_vars])
        ds = ds.isel(z_t=19,time=0)
        lon,lat,field = adjust_pop_grid(ds.TLONG.values,ds.TLAT.values,ds.O2.values)

    try:    
        extent = [120,290,-30,30]
        prj = ccrs.Mercator(central_longitude=np.mean(extent[0:2]),
                            min_latitude=extent[2],
                            max_latitude=extent[3])

        fig = plt.figure(figsize=(24,8))
        ax = plt.axes(projection=prj)
        ax.set_extent(extent)

        pc = ax.pcolormesh(lon,lat,field,
                           vmin=0.,vmax=250.,
                           cmap = cmocean.cm.delta,
                           transform=ccrs.PlateCarree())

        ax.set_rasterization_zorder(2)
        pc.set_rasterized(True)
        pc.set_zorder(1)

        land = ax.add_feature(
            cartopy.feature.NaturalEarthFeature('physical','land','110m',
            edgecolor='face',
            facecolor='black'))

        ax.set_title('O$_2$ (z = –200 m)',loc='left')
        ax.set_title(datestr,loc='right')

        cb = plt.colorbar(pc,orientation='vertical',shrink=0.8)
        cb.ax.set_title('mmol m$^{-3}$',loc='left')
        p = cb.ax.get_position()
        cb.ax.set_position([p.x0-0.02,p.y0,p.width,p.height])
        plt.savefig(plot_name,dpi=300,bbox_inches='tight')    
        plt.close()
        
    except Exception as e:
        print("error: %s" % e)

In [5]:
USER = os.environ['USER']
PROJECT = os.environ['PBS_ACCOUNT']
if True:
    from dask.distributed import Client
    from dask_jobqueue import PBSCluster

    dask.config.set({'distributed.dashboard.link':'http://localhost:{port}/status'})
    
    project = PROJECT

    cluster = PBSCluster(queue='regular',
                         cores = 9,
                         processes = 9,
                         memory = '100GB',          
                         project = project,
                         walltime = '04:00:00',
                         local_directory=f'/glade/scratch/{USER}/dask-tmp')
    client = Client(cluster)

    Nnodes = 2
    cluster.scale(9*Nnodes)

PROJECT = os.environ['PBS_ACCOUNT']
USER = os.environ['USER']
if True:
    from dask.distributed import Client
    from dask_jobqueue import PBSCluster

    project = PROJECT

    cluster = PBSCluster(queue='economy',
                         cores = 36,
                         processes = 1,
                         memory = '40GB',          
                         project = project,
                         walltime = '01:00:00',
                         local_directory=f'/glade/scratch/{USER}/dask-tmp')

cluster.scale(10)

In [13]:
!qstat -u $USER


chadmin1: 
                                                            Req'd  Req'd   Elap
Job ID          Username Queue    Jobname    SessID NDS TSK Memory Time  S Time
--------------- -------- -------- ---------- ------ --- --- ------ ----- - -----
3141893.chadmin abanihi  economy  STDIN        3332   1   1    --  02:00 R 01:34
3142015.chadmin abanihi  regular  dask-worke  68126   1   1    --  04:00 R 00:00
3142016.chadmin abanihi  regular  dask-worke  29452   1   1    --  04:00 R 00:00


In [14]:
do_delayed = True
results = []
for i in range(len(files)-1,-1,-1):
    if do_delayed:
        figo = dask.delayed(plot,pure=True)(files[i],i)
        results.append(figo)
    else:
        plot(files[i],i)
        

In [15]:
res = dask.compute(*results) 